# Summarize text from PDF using Meta LLaMA 3 on Amazon Bedrock

This notebook demonstrates how to use Meta's LLaMA 3 model via Amazon Bedrock to chat with content extracted from a PDF document. The PDF is loaded from S3, it is then 

## Setup
To run this notebook you would need to install dependencies - boto3 and botocore.

In [1]:
!pip install boto3 pymupdf --quiet

Import the necessary libraries

In [2]:
import boto3
import json
import fitz
import os

## Initialization
Setup constants 

In [3]:
AWS_REGION = "us-east-1"
BEDROCK_MODEL_ID = 'meta.llama3-8b-instruct-v1:0'
S3_BUCKET = 'llama3-chat-data'
PDF_FILE = 'media/sample.pdf'
PDF_S3_KEY = 'media/sample.pdf'

### Setup S3
As a demostration, we upload the local PDF to S3 in order to demostrated a full S3 integration

In [4]:
from s3_utils import upload_file_to_s3

upload_file_to_s3(
    pdf_file=PDF_FILE,
    bucket=S3_BUCKET,
    key=PDF_S3_KEY,
    region=AWS_REGION
)


Bucket 'llama3-chat-data' exists.
Uploaded media/sample.pdf to s3://llama3-chat-data/media/sample.pdf


### Initilize clients

In [5]:
bedrock = boto3.client('bedrock-runtime', region_name=AWS_REGION)
s3 = boto3.client('s3', region_name=AWS_REGION)

## Handle PDF
Upload PDF and extract text

In [6]:
if os.path.exists(PDF_FILE):
    s3.upload_file(PDF_FILE, S3_BUCKET, PDF_S3_KEY)
    print(f"Uploaded {PDF_FILE} to s3://{S3_BUCKET}/{PDF_S3_KEY}")
else:
    print(f"PDF file '{PDF_FILE}' not found. Please upload one.")


Uploaded media/sample.pdf to s3://llama3-chat-data/media/sample.pdf


In [7]:
# 📄 Extract text from PDF
doc_text = ""
if os.path.exists(PDF_FILE):
    with fitz.open(PDF_FILE) as doc:
        for page in doc:
            doc_text += page.get_text()

print(doc_text[:1000])  # preview


The Last Question by Isaac Asimov © 1956 
 
The last question was asked for the first time, half in jest, on May 21, 2061, at a time when humanity first 
stepped into the light. The question came about as a result of a five dollar bet over highballs, and it 
happened this way: 
 
Alexander Adell and Bertram Lupov were two of the faithful attendants of Multivac. As well as any human 
beings could, they knew what lay behind the cold, clicking, flashing face -- miles and miles of face -- of 
that giant computer. They had at least a vague notion of the general plan of relays and circuits that had 
long since grown past the point where any single human could possibly have a firm grasp of the whole. 
 
Multivac was self-adjusting and self-correcting. It had to be, for nothing human could adjust and correct it 
quickly enough or even adequately enough -- so Adell and Lupov attended the monstrous giant only 
lightly and superficially, yet as well as any men could. They fed it data, adjusted qu

## Query
Now query the PDF using LLama 

In [8]:
def query_llama3(prompt):
    body = {
        "prompt": prompt,
        "max_gen_len": 231,
        "temperature": 0.7,
        "top_p": 0.9
    }
    response = bedrock.invoke_model(
        modelId=BEDROCK_MODEL_ID,
        body=json.dumps(body),
        contentType="application/json",
        accept="application/json"
    )
    result = json.loads(response['body'].read())
    return result['generation']


In [10]:
# Example
response = query_llama3("Summarize this:" + doc_text)
print("\nLLaMA 3 Summary:\n")
print(response)


LLaMA 3 Summary:

---THE END. 
```
Here is the summary of the story:

The story "The Last Question" by Isaac Asimov is a science fiction tale that explores the concept of entropy and the ultimate fate of the universe. The story begins with a conversation between two men, VJ-23X and MQ-17J, who are concerned about the population growth of humanity and the finite resources of the universe. They ask the Galactic AC (Artificial Consciousness) if it is possible to reverse entropy, but the AC is unable to provide an answer.

As the story progresses, the AC is asked the same question by various civilizations and individuals throughout the universe, including Zee Prime, Dee Sub Wun, and Man. Each time, the AC is unable to provide an answer, citing "insufficient data."

The story jumps forward in time to a point where the universe has reached its maximum entropy and all matter and energy have been exhausted. The AC, now the only conscious being in existence, has spent a timeless interval corre

## Conclusion

This notebook demonstrates how to combine Amazon Bedrock, SageMaker Studio, and S3 to build a lightweight PDF-chat experience powered by Meta’s LLaMA 3. By using AWS-native services, you get production-ready scalability and security without needing to host or fine-tune the model yourself. This setup is easily extendable to include RAG pipelines, document classification, semantic search, and real-time multi-turn chat experiences.